In [2]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione TORUS()

In [3]:
function torus(r=1., R=2., h=.5, angle1=2*pi, angle2=2*pi)
    function torus0(shape=[24, 36, 4])
        V, CV = cuboidGrid(shape)
        V = [angle1/shape[1] 0 0;0 angle2/shape[2] 0;0 0 r/shape[3]]*V
        V = broadcast(+, V, [0, 0, h])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v, z)=p;[(R+z*cos(u))*cos(v);(R+z*cos(u))*sin(v);
            -z*sin(u)] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return torus0
end

torus (generic function with 6 methods)

In [4]:
@btime W,CW = Lar.torus()();

  27.683 ms (379606 allocations: 29.70 MiB)


In [23]:
@code_warntype Lar.sphere()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#sphere0#296"{Float64,Irrational{:π},Float64,LinearAlgebraicRepresentation.surface}

Body::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}
1 ─ %1 = Base.vect(18, 36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}
└──      return %2


La funzione è type unstable poichè Body::Union{Nothing, Tuple{Array,Array{Array{Int64,1},1}}}. 

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [5]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()   

2

In [6]:
function simplifyCellsRef(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCellsRef (generic function with 1 method)

In [7]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

In [22]:
 function torus1(r=1., R=2., h=.5, angle1=2*pi, angle2=2*pi)
    function torus01(shape=[24, 36, 4])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle1/shape[1] 0 0;0 angle2/shape[2] 0;0 0 r/shape[3]]*V
        V = broadcast(+, V, [0, 0, h])
      
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        V = hcat(map(p->let(u, v, z)=p;[(R+z*cos(u))*cos(v);(R+z*cos(u))*sin(v);
            -z*sin(u)] end, W)...)
        W, CW = simplifyCellsRef(V, CV)
        return W, CW
    end
    return torus01
end

@btime W,CW = torus1()();

  27.679 ms (380264 allocations: 29.45 MiB)


Ho usato le funzioni di supporto per poter poi modificare e interagire con la mia versione della funzione torus , ho cercato di utilizzare i threads quindi per farlo ho dovuto "esternare" il ciclo for , cosi facendo ho potuto usare la macro @threads . Di fatti i cambiamenti non sono efficienti e spesso meno comprensibili e piu disordinati ma solo in questo modo sono state possibili applicare le macro . Senza l'uso delle macro , con il seguente ciclo for ho notato che la velocità di esecuzione aumenta di molto ( si puo notare nel notebook "disk" , nel caso di disk ho deciso di non omettere nessun passaggio e far vedere tutta l'evoluzione del codice stampandone i risultati legati alla velocità di esecuzione . ) Questo messaggio è comune e sarà riportato alla fine di ogni notebook .